In [ ]:
# !python3 -m pip install firebase-admin google-cloud-firestore

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import time

In [ ]:
cred = credentials.Certificate("/Path to your Firebase SDK json/Example: firebase-adminsdk-boe4r-596dbdf4e8.json")
firebase_admin.initialize_app(cred)

store = firestore.client()

In [ ]:
CATEGORY = 'Cellphones and Accessories'
df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Cell_Phones_and_Accessories_downloaded.csv'

# CATEGORY = 'Clothing For Men'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Clothing_Men_downloaded.csv'

# CATEGORY = 'Clothing For Women'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Clothing_Women_downloaded.csv'

# CATEGORY = 'Electronics'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Electronics_downloaded.csv'

# CATEGORY = 'Home and Kitchen'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Home_and_Kitchen_downloaded.csv'

# CATEGORY = 'Pet Supplies'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Pet_Supplies_downloaded.csv'

# CATEGORY = 'Shoes'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Shoes_downloaded.csv'

# CATEGORY = 'Watches'
# df_path = '../csv/filtered_cat_over_500items/downloaded/data-v4/5000/Watches_downloaded.csv'

In [ ]:
df = pd.read_csv(df_path)
df.drop(columns=['related','salesRank', 'categories', 'description', 'timestamp', 'imUrl_local'], inplace=True)
df.info()

In [ ]:
df.sample()

In [ ]:
# test first
doc_ref = store.collection(u'products_amazon')

doc_ref.add({u'product_id': df.iloc[644].asin, 
             u'title': df.iloc[644].title, 
             u'price': df.iloc[644].price,
             u'imUrl': df.iloc[644].imUrl,
             u'brand': [df.iloc[644].brand if pd.notna(df.iloc[644].brand) else ''],
             u'rating': df.iloc[644].rating,
             u'created_at': int(time.time())
            })

In [ ]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
def import_to_firestore(df, category, collection_name):
    data = []
    headers = list(df)
    line_count = 0
    
    for index, row in df.iterrows():
        obj = {}
        
#         if index == 1: break
        for idx, item in enumerate(row):
            obj[headers[idx]] = item
            
        obj['category'] = category
        obj['created_at'] = int(time.time()) + index
            
        data.append(obj)
        line_count += 1
        print(f'Processed {line_count} lines.')
        
    for batched_data in batch_data(data, 499):
        batch = store.batch()
        for data_item in batched_data:
            doc_ref = store.collection(collection_name).document()
            batch.set(doc_ref, data_item)
        batch.commit()

In [ ]:
COLLECTION_NAME = 'products_amazon'

In [ ]:
df.sample()

In [ ]:
import_to_firestore(df, CATEGORY, COLLECTION_NAME)